In [1]:
import requests
import math
import pandas as pd
import numpy as np
import itertools
import math
import random
import matplotlib.pyplot as plt
from matplotlib.pylab import mpl
import folium
from folium.plugins import HeatMap
import warnings
warnings.filterwarnings('ignore')

In [54]:
# get LAT and LNG for LA county Zip Codes
x = pd.read_csv('zip_codes.txt')
x = x[[str(i)[0] == '9' for i in x['ZIP']]].copy()
x = x[[len(str(i)) >= 5 for i in x['ZIP']]].copy()
y = pd.read_csv('LA_County_ZIP_Codes.csv')
y = y.iloc[:,1:2].rename(columns = {'ZIPCODE':'ZIP'})
x = pd.merge(y, x, how='left', on='ZIP')
x = x.dropna()
x

,ZIP,LAT,LNG
0,90001,33.974027,-118.249509
1,90002,33.949099,-118.246737
2,90003,33.964131,-118.272783
3,90004,34.076198,-118.310722
4,90005,34.059163,-118.306892
...,...,...,...
308,90090,34.072862,-118.240989
309,91709,33.963723,-117.738596
310,91748,33.974883,-117.900676
311,91765,33.986270,-117.821282


In [3]:
# distance calculate functions
def rad(x):
  return x * math.pi / 180.0;

def getHaversineDistance(p1, p2):
  R = 6378137; # Earth’s mean radius in meter
  dLat = rad(p2.iloc[:,0] - p1.iloc[:,0]);
  dLong = rad(p2.iloc[:,1] - p1.iloc[:,1]);
  a = (np.sin(dLat / 2) * np.sin(dLat / 2) +
    np.cos(rad(p1.iloc[:,0])) * np.cos(rad(p2.iloc[:,0])) *
    np.sin(dLong / 2) * np.sin(dLong / 2))
  c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))
  d = R * c #
  return d # // returns the distance in meter
def Meters2Miles(meters):
    return meters * 0.000621371

def Miles2Meters(miles):
    return miles * 1609.34

In [35]:
# income data
data2 = pd.read_excel('tax_data2.xlsx')
data2 = data2.dropna().reset_index()
data2['ZIP code'] = data2['ZIP code'].astype(int)
rich_number = data2[data2['Size of AGI'] == '200,000 or more'][['ZIP code','Number of returns']]
rich_number = rich_number[1:-1]
rich_number.rename(columns={'ZIP code': 'ZIP'}, inplace=True)
data1 = pd.merge(x, rich_number, how='left', on='ZIP')
data1 = data1.dropna().reset_index(drop=True)
data1

,ZIP,LAT,LNG,Number of returns
0,90001,33.974027,-118.249509,20.0
1,90002,33.949099,-118.246737,20.0
2,90003,33.964131,-118.272783,40.0
3,90004,34.076198,-118.310722,1570.0
4,90005,34.059163,-118.306892,550.0
...,...,...,...,...
281,90013,34.045405,-118.240454,360.0
282,91709,33.963723,-117.738596,3610.0
283,91748,33.974883,-117.900676,980.0
284,91765,33.986270,-117.821282,2120.0


In [110]:
data2 = pd.read_excel('Data_MQE2023.xlsx')
data2
data2['zip'] = data2['zip'].astype(int)
# rich_number = data2[data2['Size of AGI'] == '200,000 or more'][['ZIP code','Number of returns']]
# rich_number = rich_number[1:-1]
data2.rename(columns={'zip': 'ZIP'}, inplace=True)
# data1 = pd.merge(x, data2, how='left', on='ZIP')
# data1 = data1.dropna().reset_index(drop=True)
# data1
data2

,ZIP,avg_houshold_income,pct_above300k,count
0,90001,132203,0.08,720
1,90002,131440,0.07,861
2,90003,128170,0.06,1079
3,90004,138075,0.07,658
4,90005,156520,0.10,495
...,...,...,...,...
1003,93561,168593,0.10,412
1004,93562,134164,0.17,6
1005,93581,374876,0.80,5
1006,93584,70800,0.00,1


In [8]:
data0 = pd.read_excel('tax_data2.xlsx')
total_number = data0[data0['Size of AGI'] == 'Total'][['ZIP code','Number of returns']]
total_number = total_number[1:-1]
total_number
total_number.rename(columns={'ZIP code': 'ZIP'}, inplace=True)
total_number
data3 = pd.merge(x, total_number, how='left', on='ZIP')
data3 = data3.dropna().reset_index(drop=True)
data3.rename(columns={'Number of returns': 'Number of Residents'}, inplace=True)
del data3['LAT']
del data3['LNG']
data3

,ZIP,Number of Residents
0,90001,24730.0
1,90002,22730.0
2,90003,31100.0
3,90004,29020.0
4,90005,17710.0
...,...,...
281,90013,5200.0
282,91709,38440.0
283,91748,23220.0
284,91765,23420.0


In [25]:
# Payroll data
import requests

url = "https://api.census.gov/data/2018/zbp"
params = {
    'get': 'EMP',
    'for': 'zipcode:*',  # Use '*' to request data for all ZIP codes
}

response = requests.get(url, params=params)
data = response.json()
df = pd.DataFrame(data)
df.columns = df.iloc[0]
df = df.iloc[1:,:]
df['EMP'] = df['EMP'].astype('int')
df = df[df['EMP'] != 0]
df.rename(columns={'zip code': 'ZIP'}, inplace=True)
df['ZIP'] = df['ZIP'].astype('int')
data3 = pd.merge(data2, df, how='left', on='ZIP')
data3 = data3.dropna().reset_index(drop=True)
data3

,ZIP,avg_houshold_income,pct_above300k,count,EMP
0,90001,132203,0.08,720,6673.0
1,90002,131440,0.07,861,1842.0
2,90003,128170,0.06,1079,6188.0
3,90004,138075,0.07,658,8329.0
4,90005,156520,0.10,495,6096.0
...,...,...,...,...,...
933,93561,168593,0.10,412,4394.0
934,93562,134164,0.17,6,571.0
935,93581,374876,0.80,5,42.0
936,93584,70800,0.00,1,27.0


In [55]:
data = pd.merge(x, data3, how='left', on='ZIP')
data = data.dropna().reset_index(drop=True)
data

,ZIP,LAT,LNG,avg_houshold_income,pct_above300k,count,EMP
0,90001,33.974027,-118.249509,132203.0,0.08,720.0,6673.0
1,90002,33.949099,-118.246737,131440.0,0.07,861.0,1842.0
2,90003,33.964131,-118.272783,128170.0,0.06,1079.0,6188.0
3,90004,34.076198,-118.310722,138075.0,0.07,658.0,8329.0
4,90005,34.059163,-118.306892,156520.0,0.10,495.0,6096.0
...,...,...,...,...,...,...,...
289,90013,34.045405,-118.240454,124747.0,0.01,68.0,15146.0
290,91709,33.963723,-117.738596,174811.0,0.11,1101.0,14795.0
291,91748,33.974883,-117.900676,136232.0,0.07,431.0,30594.0
292,91765,33.986270,-117.821282,154415.0,0.09,461.0,13219.0


In [64]:
# data2.dtypes

ZIP                      int64
LAT                    float64
LNG                    float64
Number of returns      float64
EMP                      int32
Number of Residents    float64
dtype: object

In [29]:
df = data.copy()

# Convert income column to numeric
df['EMP'] = pd.to_numeric(df['EMP'])

# Create a new column with categorical labels
# bins = [0, df['EMP'].quantile(0.33), df['EMP'].quantile(0.66), float('inf')]
# labels = ['lower income', 'medium income', 'higher income']
# df['income_category'] = pd.cut(df['EMP'], bins=bins, labels=labels, right=False)

# Create a folium map centered around the mean of latitudes and longitudes
map_center = [df['LAT'].mean(), df['LNG'].mean()]
mymap = folium.Map(location=map_center, zoom_start=10)

# Create HeatMap with ZIP code coordinates and income category values
heat_data = [[row['LAT'], row['LNG'], row['EMP']] for index, row in df.iterrows()]
HeatMap(heat_data, gradient={0.0: 'blue', 0.33: 'lime', 0.66: 'red'}, min_opacity=0.5, max_opacity=0.8, radius=15, blur=10).add_to(mymap)

# Add a larger legend
legend_html = '''
     <div style="position: fixed; bottom: 50px; left: 50px; width: 180px; height: 100px; 
                 background-color: white; border:2px solid grey; z-index:9999; font-size:14px;">
     &nbsp; <b>Legend</b> <br>
     &nbsp; 0 - <span style="color: blue;">Lower Presence</span><br>
     &nbsp; 0.33 - <span style="color: lime;">Medium Presence</span><br>
     &nbsp; 0.66 - <span style="color: red;">Higher Presence</span>
      </div>
     '''

mymap.get_root().html.add_child(folium.Element(legend_html))

# Save the map to an HTML file
mymap

In [65]:
min_value = data['count'].min()
max_value = data['count'].max()
data['count'] = (data['count'] - min_value) / (max_value - min_value)
min_value = data['EMP'].min()
max_value = data['EMP'].max()
data['EMP'] = (data['EMP'] - min_value) / (max_value - min_value)
# min_value = data['Number of Residents'].min()
# max_value = data['Number of Residents'].max()
# data['Number of Residents'] = (data['Number of Residents'] - min_value) / (max_value - min_value)
data

,ZIP,LAT,LNG,avg_houshold_income,pct_above300k,count,EMP
0,90001,33.974027,-118.249509,132203.0,0.08,0.490116,0.082051
1,90002,33.949099,-118.246737,131440.0,0.07,0.586230,0.022056
2,90003,33.964131,-118.272783,128170.0,0.06,0.734833,0.076028
3,90004,34.076198,-118.310722,138075.0,0.07,0.447853,0.102617
4,90005,34.059163,-118.306892,156520.0,0.10,0.336742,0.074885
...,...,...,...,...,...,...,...
289,90013,34.045405,-118.240454,124747.0,0.01,0.045671,0.187276
290,91709,33.963723,-117.738596,174811.0,0.11,0.749830,0.182917
291,91748,33.974883,-117.900676,136232.0,0.07,0.293115,0.379121
292,91765,33.986270,-117.821282,154415.0,0.09,0.313565,0.163345


In [10]:
data2

,ZIP,LAT,LNG,Number of returns,EMP,Number of Residents
0,90001,33.974027,-118.249509,0.003317,0.082051,0.487098
1,90002,33.949099,-118.246737,0.003317,0.022056,0.447400
2,90003,33.964131,-118.272783,0.006633,0.076028,0.613537
3,90004,34.076198,-118.310722,0.260365,0.102617,0.572251
4,90005,34.059163,-118.306892,0.091211,0.074885,0.347757
...,...,...,...,...,...,...
295,90013,34.045405,-118.240454,0.059701,0.187276,0.099444
296,91709,33.963723,-117.738596,0.598673,0.182917,0.759230
297,91748,33.974883,-117.900676,0.162521,0.379121,0.457126
298,91765,33.986270,-117.821282,0.351575,0.163345,0.461096


In [82]:
dis_matrix = pd.DataFrame()
for i in range(len(data)):
    p1 = pd.DataFrame([data.iloc[i, :][['LAT', 'LNG']]] * len(data)).reset_index(drop=True)
    p2 = pd.DataFrame(data[['LAT', 'LNG']]).reset_index(drop=True)
    dist_meters = getHaversineDistance(p1,p2)
    dist_miles = Meters2Miles(dist_meters)
    dis_matrix[i] = dist_miles
dis_matrix

,0,1,2,3,4,5,6,7,8,9,...,284,285,286,287,288,289,290,291,292,293
0,0.000000,1.731606,1.500382,7.890536,6.745620,5.724495,4.255217,6.092842,7.185930,2.346707,...,15.967893,16.536392,16.536392,16.536392,30.511686,4.964491,29.317729,20.009959,24.577039,28.682780
1,1.731606,0.000000,1.820517,9.526288,8.358195,7.364775,5.886945,7.097721,8.767594,4.069364,...,14.239546,14.911419,14.911419,14.911419,32.130543,6.671291,29.171860,19.933719,24.542078,28.865574
2,1.500382,1.820517,0.000000,8.051132,6.858193,5.932340,4.480364,5.277822,7.216342,3.079614,...,15.555813,16.533305,16.533305,16.533305,30.561823,5.919544,30.645994,21.359115,25.944145,30.116913
3,7.890536,9.526288,8.051132,0.000000,1.198585,2.166139,3.641076,5.050709,1.015944,5.634543,...,23.604288,24.423770,24.423770,24.423770,22.621244,4.555284,33.710898,24.529438,28.737917,31.868725
4,6.745620,8.358195,6.858193,1.198585,0.000000,1.059635,2.491421,4.121806,0.555373,4.539965,...,22.408684,23.265510,23.265510,23.265510,23.774607,3.924673,33.246581,24.008350,28.292778,31.587553
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
289,4.964491,6.671291,5.919544,4.555284,3.924673,3.084450,2.810310,6.577528,4.475527,2.848822,...,20.809893,20.968677,20.968677,20.968677,26.466486,0.000000,29.326921,20.083680,24.378572,27.761458
290,29.317729,29.171860,30.645994,33.710898,33.246581,32.387080,31.640459,35.022361,33.792747,29.979873,...,30.894720,26.701292,26.701292,26.701292,51.255137,29.326921,0.000000,9.329842,4.992864,5.526240
291,20.009959,19.933719,21.359115,24.529438,24.008350,23.122819,22.334659,25.693152,24.558586,20.652511,...,23.724816,20.076650,20.076650,20.076650,43.284858,20.083680,9.329842,0.000000,4.621513,9.510714
292,24.577039,24.542078,25.944145,28.737917,28.292778,27.447322,26.737115,30.175242,28.836836,25.124963,...,27.752604,23.847850,23.847850,23.847850,46.451878,24.378572,4.992864,4.621513,0.000000,5.383069


In [131]:


mpl.rcParams['font.sans-serif'] = ['SimHei']  # 添加这条可以让图形显示中文

def logit(x):
    return 1 / (1 + np.exp(-0.1*(x-15)))

def logit2(x):
    return 1 / (1 + np.exp(0.7*(x-4)))

# object function
def calFitness(chrom,dis_matrix, cnum, d, r, p):
    
    declist = []
    dec_chrom =chrom.copy()
    demand = [0,0,0,0,0,0,0]
    supply = [0,0,0,0,0,0,0]
    dsd = []
    
    for i in range(cnum):
        if chrom[i] == 1:
            declist.append(i)

    for i in range(cnum):
        min_dis = 100
        k = p+1
        for j in range(p):
            if dis_matrix.loc[i,declist[j]] <= min_dis:
                min_dis = dis_matrix.loc[i,declist[j]]
                k = j
        demand[k] += d[i]*logit2(min_dis)
        supply[k] += s[i]*logit2(min_dis)
    for i in range(p):
        for j in range(p):
            if i!= j:
                dsd.append(min(demand[i],supply[j])*logit(dis_matrix[declist[i]][declist[j]]))
    total_demand = sum(dsd)
    return total_demand #覆盖最大

def traversal_search(chrom,dis_matrix,tabu_list, cnum, d, p):
    #邻域随机遍历搜索
    traversal = 0#搜索次数
    traversal_list = []#存储局部搜索生成的解,也充当局部禁忌表
    traversal_value = []#存储局部解对应路径距离
    while traversal <= traversalMax:
        new_chrom = chrom.copy()#复制当前路径，并交换生成新路径
        pos1,pos2 = random.randint(0, cnum-1),random.randint(0, cnum-1)#交换点
        if new_chrom[pos1] != new_chrom[pos2]:
            new_chrom[pos1],new_chrom[pos2]=new_chrom[pos2],new_chrom[pos1]
            if (new_chrom not in traversal_list) & (new_chrom not in tabu_list):
                new_value = calFitness(new_chrom,dis_matrix, cnum, d, r, p)
                traversal_list.append(new_chrom)
                traversal_value.append(new_value)
                traversal += 1
            new_value = calFitness(new_chrom,dis_matrix, cnum, d, r, p)#当前路径距离
            #新生成路径不在全局禁忌表和局部禁忌表中，为有效搜索，否则继续搜索
    
    return max(traversal_value),traversal_list[traversal_value.index(max(traversal_value))]
    
    
def initialize(cnum , p):
    """
    in:dnum-需求点数量，cnum-备选点数量，
    out:染色体
    """
    clist = random.sample(range(cnum),p)
    cchrom = [1 if i in clist else 0 for i in range(cnum) ]
    
    chrom = cchrom
    
    return chrom
    
    
#画散点图
def draw_sca(demandCoordinates):
    coordinates = demandCoordinates

    # Create a base map
    map_object = folium.Map(location=[coordinates[0][0], coordinates[0][1]], zoom_start=4)

    # Add markers for each coordinate
    for coord in coordinates:
        folium.Marker(coord).add_to(map_object)
#         Circle(
#         location=coord,
#         radius=radius,
#         color='blue',
#         fill=True,
#         fill_color='blue',
#         fill_opacity=0.2,
#         popup='Your Circle'
#     ).add_to(map_object)

    # Save the map to an HTML file
    return map_object


#画分布图
def draw_path(chrom, demandCoordinates, centerCoordinates, p, cnum, r):
    from folium import Circle
    centerlist = []
    for i in range(cnum):
        if chrom[i] == 1:
            centerlist.append(centerCoordinates[i])
    map_object = draw_sca(demandCoordinates)
#     for i in range(cnum,len(chrom)):
#         if dis_matrix.loc[i-cnum,chrom[i]]<=r:#检测时候满足服务半径约束，不满足则无需连接
#             if chrom[i] == 1:
#                 plt.plot([centerlist[0][0],demandCoordinates[i-cnum][0]], [centerlist[0][1],demandCoordinates[i-cnum][1]], 'r-', color='#4169E1', alpha=0.8, linewidth=0.8)#plt.plot([x1,x2],[y1,y2])
#             elif chrom[i] == 2:
#                 plt.plot([centerlist[1][0],demandCoordinates[i-cnum][0]], [centerlist[1][1],demandCoordinates[i-cnum][1]], 'r-', color='#4169E1', alpha=0.8, linewidth=0.8)#plt.plot([x1,x2],[y1,y2])
#             elif chrom[i] == 3:
#                 plt.plot([centerlist[2][0],demandCoordinates[i-cnum][0]], [centerlist[2][1],demandCoordinates[i-cnum][1]], 'r-', color='#4169E1', alpha=0.8, linewidth=0.8)#plt.plot([x1,x2],[y1,y2])
#     draw_sca(demandCoordinates,centerCoordinates)

    radius = Miles2Meters(r)
    # Create a base map

    # Add markers for each coordinate
    for coord in centerlist:
        folium.Marker(coord).add_to(map_object)
        Circle(
        location=coord,
        radius=radius,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.2,
        popup='Your Circle'
    ).add_to(map_object)

    # Save the map to an HTML file
    return map_object



if __name__ == '__main__':
    #参数

    #TS参数
    tabu_limit = 1000 #禁忌长度
    iterMax = 100#迭代次数
    traversalMax = 500#每一代局部搜索次数
    
    tabu_list = [] #禁忌表
    tabu_time = [] #禁忌次数
    best_value = math.pow(10,10)#较大的初始值，存储最优解
    best_line = []#存储最优路径
    
    
    #需求点位置及需求量，备选中心位置及能力
    demandCoordinates = data[['LAT','LNG']].values.tolist()
    centerCoordinates = demandCoordinates
    d = data['count'].values.tolist()#需求量，对应demandCoordinates
    s = data['EMP'].values.tolist()#供给量
    
    #draw_sca(demandCoordinates)#位置图
    
    p = 6 #待决策物流中心数量
    cnum = len(centerCoordinates) #中心数量
    
    r = 5 #备选中心服务半径
    
    
    # 计算中心与需求点之间的距离
#     dis_matrix = pd.DataFrame(data=None,columns=range(cnum),index=range(dnum))
#     for i in range(dnum):
#         xi,yi = demandCoordinates[i][0],demandCoordinates[i][1]
#         for j in range(len(centerCoordinates)):
#             xj,yj = centerCoordinates[j][0],centerCoordinates[j][1]
#             dis_matrix.iloc[i,j] = round(math.sqrt((xi-xj)**2+(yi-yj)**2),2)
    
    
    # 初始化,随机构造
    num = 1000
    chroms = [initialize(cnum, p) for i in range(num)]
    values = [calFitness(chrom,dis_matrix, cnum, d, r, p) for chrom in chroms]
    
    best_value = max(values)
    best_chrom = chroms[values.index(best_value)]
    chrom,value = best_chrom,best_value
    
    # #存储当前最优
    print('初代最优值 %.1f' % (best_value))
    
    best_value_list = []
    best_value_list.append(best_value)
    #更新禁忌表
    tabu_list.append(best_chrom)
    tabu_time.append(tabu_limit)
    
    itera = 0
    while itera <= iterMax:
        new_value,new_chrom = traversal_search(chrom,dis_matrix,tabu_list, cnum, d, p)
        if new_value > best_value:#优于最优解
            best_value,best_chrom = new_value,new_chrom#更新最优解
            best_value_list.append(best_value)
            count = 0
        else:
            count += 1
        print('第%.d代最优值 %.1f' % (itera,best_value))
        chrom,value = new_chrom,new_value#更新当前解
        if count == 10:
            print("迭代完成")
            break
        
        #更新禁忌表
        tabu_time = [x-1 for x in tabu_time]
        if 0 in tabu_time:
            tabu_list.remove(tabu_list[tabu_time.index(0)])
            tabu_time.remove(0)
        
        tabu_list.append(chrom)
        tabu_time.append(tabu_limit)
        itera +=1
    
    map_object = draw_path(best_chrom, demandCoordinates, centerCoordinates, p, cnum, r)
    map_object.save('optimal_location2.html')
    map_object

初代最优值 26.3
第0代最优值 27.7
第1代最优值 29.1
第2代最优值 30.1
第3代最优值 30.6
第4代最优值 31.0
第5代最优值 31.5
第6代最优值 31.6
第7代最优值 31.6
第8代最优值 32.2
第9代最优值 32.2
第10代最优值 32.2
第11代最优值 32.2
第12代最优值 32.2
第13代最优值 32.2
第14代最优值 32.2
第15代最优值 32.2
第16代最优值 32.2
第17代最优值 32.2
第18代最优值 32.2
第19代最优值 32.2
第20代最优值 32.2
第21代最优值 32.2
第22代最优值 32.2
第23代最优值 32.3
第24代最优值 32.3
第25代最优值 32.3
第26代最优值 32.3
第27代最优值 32.3
第28代最优值 32.3
第29代最优值 32.3
第30代最优值 32.3
第31代最优值 32.3
第32代最优值 32.3
第33代最优值 32.3
迭代完成


In [132]:
data[np.array(best_chrom)==1]

,ZIP,LAT,LNG,avg_houshold_income,pct_above300k,count,EMP
52,90063,34.045105,-118.185914,130399.0,0.06,0.481936,0.046893
67,90212,34.062210,-118.401966,265538.0,0.31,0.034765,0.202899
83,90260,33.888560,-118.351813,144439.0,0.07,0.228357,0.055723
153,90813,33.782259,-118.196793,138622.0,0.09,0.407635,0.144220
221,91405,34.199824,-118.447609,131004.0,0.07,0.353783,0.172522
263,91790,34.066964,-117.937007,137947.0,0.06,0.378323,0.178843


In [125]:
def min_max_normalize(input_list):
    """
    Normalize a list using min-max scaling.
    
    Parameters:
        input_list (list): The input list of numeric values.
        
    Returns:
        list: The normalized list.
    """
    # Check if the list is not empty
    if not input_list:
        raise ValueError("Input list is empty")

    # Find the minimum and maximum values
    min_val = min(input_list)
    max_val = max(input_list)

    # Check if the range is not zero to avoid division by zero
    if min_val == max_val:
        raise ValueError("Minimum and maximum values are the same, causing division by zero")

    # Normalize the list using min-max scaling
    normalized_list = [(x - min_val) / (max_val - min_val) for x in input_list]

    return normalized_list

In [126]:
len(best_chrom)

294

In [134]:
import seaborn as sns
def draw_path(chrom, demandCoordinates, centerCoordinates, p, cnum, r):
    from folium import Circle
    centerlist = []
    coverlist = []
    declist = []
    dslist = []
    dsd = []
    demand = [0,0,0,0,0,0,0]
    supply = [0,0,0,0,0,0,0]
    for i in range(cnum):
        if chrom[i] == 1:
            centerlist.append(centerCoordinates[i])
            declist.append(i)
    for i in range(cnum):
        k = p+1
        min_dis = 100
        for j in range(p):
            if dis_matrix.loc[i,declist[j]] <= min_dis:
                min_dis = dis_matrix.loc[i,declist[j]]
                k = j
        dslist.append(min(d[i]*logit2(min_dis),s[i]*logit2(min_dis)))
        coverlist.append(k)
        demand[k] += d[i]*logit2(min_dis)
        supply[k] += s[i]*logit2(min_dis)
    for i in range(p):
        for j in range(p):
            if i!= j:
                dsd.append((demand[i]+supply[j])*logit(dis_matrix[declist[i]][declist[j]]))
    dslist = min_max_normalize(dslist)
#     print(coverlist)
#     print(coverlist)
#     for i in range(cnum,len(chrom)):
#         if dis_matrix.loc[i-cnum,chrom[i]]<=r:#检测时候满足服务半径约束，不满足则无需连接
#             if chrom[i] == 1:
#                 plt.plot([centerlist[0][0],demandCoordinates[i-cnum][0]], [centerlist[0][1],demandCoordinates[i-cnum][1]], 'r-', color='#4169E1', alpha=0.8, linewidth=0.8)#plt.plot([x1,x2],[y1,y2])
#             elif chrom[i] == 2:
#                 plt.plot([centerlist[1][0],demandCoordinates[i-cnum][0]], [centerlist[1][1],demandCoordinates[i-cnum][1]], 'r-', color='#4169E1', alpha=0.8, linewidth=0.8)#plt.plot([x1,x2],[y1,y2])
#             elif chrom[i] == 3:
#                 plt.plot([centerlist[2][0],demandCoordinates[i-cnum][0]], [centerlist[2][1],demandCoordinates[i-cnum][1]], 'r-', color='#4169E1', alpha=0.8, linewidth=0.8)#plt.plot([x1,x2],[y1,y2])
#     draw_sca(demandCoordinates,centerCoordinates)
    radius = Miles2Meters(r)
    # Create a base map
    map_object = folium.Map(location=[centerCoordinates[0][0], centerCoordinates[0][1]], zoom_start=10)
#     for coord in centerCoordinates:
#         folium.Marker(coord).add_to(map_object)
    # Add markers for each coordinate
    r = 5
    radius = Miles2Meters(r)
    for coord in centerlist:
        folium.Marker(coord).add_to(map_object)
        Circle(
        location=coord,
        radius=radius,
        color='blue',
        fill=True,
        fill_color='blue',
        fill_opacity=0.2,
        popup='Your Circle'
    ).add_to(map_object)
    lines_coordinates = []
    lines_coordinates2 = []
#     for i in range(cnum):
#         lines_coordinates.append([[centerCoordinates[i][0], centerCoordinates[i][1]],[centerlist[coverlist[i]][0],centerlist[coverlist[i]][1]]])
    for i in range(len(lines_coordinates)):
        folium.PolyLine(locations=lines_coordinates[i], color='red',opacity=dslist[i]*5).add_to(map_object)
    for i in range(p):
        for j in range(p):
            if i != j:
                lines_coordinates2.append([[centerlist[i][0], centerlist[i][1]],[centerlist[j][0], centerlist[j][1]]])
#     for i in range(len(lines_coordinates2)):
#         folium.PolyLine(locations=lines_coordinates2[i], color='blue',opacity=dsd[i]*0.07).add_to(map_object)
#     Save the map to an HTML file
#     map_object.save('optimal_location_v3.html')
    return map_object
map_object = draw_path(best_chrom, demandCoordinates, centerCoordinates, p, cnum, r)
map_object